In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/archive (4).zip')
df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [ ]:
df.isnull().sum()

,0
gender,0
race/ethnicity,0
parental level of education,0
lunch,0
test preparation course,0
math score,0
reading score,0
writing score,0


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline



In [ ]:
df.head(1)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74


In [ ]:
df.value_counts('test preparation course')

,count
test preparation course,
none,642
completed,358


In [ ]:
X=df.drop(columns=['test preparation course'])
y=df['test preparation course']
X

,gender,race/ethnicity,parental level of education,lunch,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,72,72,74
1,female,group C,some college,standard,69,90,88
2,female,group B,master's degree,standard,90,95,93
3,male,group A,associate's degree,free/reduced,47,57,44
4,male,group C,some college,standard,76,78,75
...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,88,99,95
996,male,group C,high school,free/reduced,62,55,55
997,female,group C,high school,free/reduced,59,71,65
998,female,group D,some college,standard,68,78,77


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
#identify cols
nums_cols=['math score','reading score','writing score']
cat_cols=['gender','race/ethnicity','parental level of education','lunch']

In [ ]:
nums_cols

['math score', 'reading score', 'writing score']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
nums_trans=Pipeline(steps=[
    ('scaler',MinMaxScaler())
])

In [ ]:
cat_trans=Pipeline(steps=[
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])


In [ ]:
preprocessor=ColumnTransformer(
    transformers=[
        ('nums',nums_trans,nums_cols),
        ('cat',cat_trans,cat_cols)
    ]
)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
pipe=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('selector',SelectKBest(score_func=chi2,k='all')),
    ('classifier',RandomForestClassifier(random_state=42))
])

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('nums',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['math score',
                                                   'reading score',
                                                   'writing score']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['gender', 'race/ethnicity',
                                                   'parental level of '
                                                   'education',
                                                   'lunch'])])),
                ('selector',
                 SelectKBest(k='all',
                             score_func=<function chi2 at 0x7d1339e28360>)),
                ('classifier', RandomForestClassifier(random_state=42))])

In [ ]:
print(pipe.score(X_test,y_test))

0.66


In [ ]:
print(y.value_counts(normalize=True))

pipe.fit(X_train, y_train)
print("Test accuracy:", pipe.score(X_test, y_test))


test preparation course
none         0.642
completed    0.358
Name: proportion, dtype: float64
Test accuracy: 0.66


In [ ]:
print(y.value_counts(normalize=True))


test preparation course
none         0.642
completed    0.358
Name: proportion, dtype: float64


In [ ]:
# Fit the pipeline on training data
pipe.fit(X_train, y_train)

# Now predict on test data
y_pred = pipe.predict(X_test)

# Evaluate
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

   completed       0.53      0.44      0.48        72
        none       0.71      0.78      0.75       128

    accuracy                           0.66       200
   macro avg       0.62      0.61      0.62       200
weighted avg       0.65      0.66      0.65       200

